In [1]:
import pandas as pd
import numpy as np
import gzip
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
Source : http://jmcauley.ucsd.edu/data/amazon/qa/ 
"""
def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 
        
def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

In [3]:
fname = 'qa_Appliances.json.gz'
fname = 'qa_Beauty.json.gz'
# fname = 'qa_Electronics.json.gz'

In [4]:
df = getDF(fname)
df = df[['questionType','question']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42422 entries, 0 to 42421
Data columns (total 2 columns):
questionType    42422 non-null object
question        42422 non-null object
dtypes: object(2)
memory usage: 994.3+ KB


In [6]:
from collections import Counter
Counter(df.questionType)

Counter({'yes/no': 18091, 'open-ended': 24331})

In [11]:
yn = Counter(df.questionType)['yes/no']
oe = Counter(df.questionType)['open-ended']
p = yn*100/(yn+oe)
print("yes/no : {0:.2f}%\topen-ended : {1:.2f}%".format(p, 100-p))

yes/no : 42.65%	open-ended : 57.35%


In [12]:
X = df.question
Y = df.questionType

In [13]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15, random_state=42)

Processing the data
- Tokenize and convert text to seq
- add padding all seq has same shape
- choose arbitrary max_len. many ways exist

In [15]:
max_words = 1000#20000
max_len = 100#len(max(X))//2
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

In [16]:
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [17]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [18]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [19]:
model = RNN()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [20]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [21]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 28846 samples, validate on 7212 samples
Epoch 1/10
28846/28846 [==============================] - 35s 1ms/step - loss: 0.4117 - acc: 0.8220 - val_loss: 0.3235 - val_acc: 0.8740
Epoch 2/10
28846/28846 [==============================] - 32s 1ms/step - loss: 0.2912 - acc: 0.8846 - val_loss: 0.2887 - val_acc: 0.8874
Epoch 3/10
28846/28846 [==============================] - 32s 1ms/step - loss: 0.2600 - acc: 0.8968 - val_loss: 0.2715 - val_acc: 0.8959
Epoch 4/10
28846/28846 [==============================] - 32s 1ms/step - loss: 0.2375 - acc: 0.9046 - val_loss: 0.2521 - val_acc: 0.8959
Epoch 5/10
28846/28846 [==============================] - 33s 1ms/step - loss: 0.2145 - acc: 0.9109 - val_loss: 0.2519 - val_acc: 0.8979
Epoch 6/10
28846/28846 [==============================] - 32s 1ms/step - loss: 0.1950 - acc: 0.9211 - val_loss: 0.2323 - val_acc: 0.9039
Epoch 7/10
28846/28846 [==============================] - 32s 1ms/step - loss: 0.1774 - acc: 0.9295 - val_loss: 0.2317 - val_acc:

In [22]:
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

6364/6364 [==============================] - 3s 476us/step
Test set
  Loss: 0.227
  Accuracy: 0.923
